# Engenharia de Features

Este notebook tem como objetivo transformar os dados consolidados em variáveis úteis para o modelo de regressão.

**Principais Transformações:**
* **Cálculo de Cubagem:** Determinação do peso cúbico e do peso final.
* **Cálculo de Distância:** Aplicação da fórmula de Haversine para obter a distância entre vendedor e cliente.
* **Seleção Final:** Filtragem das colunas, mantendo apenas as *features* essenciais e a variável *target*.

In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
import sys
import os

project_root = os.path.abspath("../../..")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.T3.regressao.features import calcular_peso_cubado, haversine_vectorized

In [2]:
BASE_PATH = Path("../../../")
DATA_INTERIM = BASE_PATH / "data" / "interim" / "T3" / "regressao"
DATA_PROCESSED = BASE_PATH / "data" / "processed" / "T3" / "regressao"

In [3]:
dataset_merged = pd.read_csv(DATA_INTERIM / "dataset_merged.csv")
dataset_merged.head()

,order_id,product_id,seller_id,price,freight_value,customer_id,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_state,seller_zip_code_prefix,seller_state,geolocation_zip_code_prefix_x,lat_customer,lng_customer,geolocation_zip_code_prefix_y,lat_seller,lng_seller
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,3ce436f183e68e07877b285a838db11a,650.0,28.0,9.0,14.0,28013,RJ,27277,SP,28013.0,-21.762775,-41.309633,27277.0,-22.496953,-44.127492
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,30000.0,50.0,30.0,40.0,15775,SP,3471,SP,15775.0,-20.220527,-50.903424,3471.0,-23.565096,-46.518565
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,3050.0,33.0,13.0,33.0,35661,MG,37564,MG,35661.0,-19.870305,-44.593326,37564.0,-22.262584,-46.171124
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,200.0,16.0,10.0,15.0,12952,SP,14403,SP,12952.0,-23.089925,-46.611654,14403.0,-20.553624,-47.387359
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,3750.0,35.0,40.0,30.0,13226,SP,87900,PR,13226.0,-23.243402,-46.827614,87900.0,-22.929384,-53.135873


## Volume e Densidade

In [4]:
df = dataset_merged.copy()
df['volume_cm3'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']
df['product_density'] = df['product_weight_g'] / (df['volume_cm3'])

## Peso Final

No transporte de cargas, o frete é cobrado com base no maior valor entre o **peso físico** real e o **peso cubado**.

**Fórmula utilizada:**
$$Peso\ Cubado\ (kg) = \frac{Comprimento \times Largura \times Altura}{Fator\ de\ Cubagem}$$

**Critérios adotados:**
* **Fator de Cubagem (6000):** É o padrão oficial da IATA e utilizado pela maioria das transportadoras rodoviárias e Correios no Brasil para converter volume ($cm^3$) em peso taxável ($kg$).
* **Conversão de Unidade:** A fórmula original resulta em quilogramas. Como a coluna de peso original (`product_weight_g`) está em gramas, multiplicamos o resultado do peso cúbico por **1000** para manter a consistência das unidades.
* **Peso Final:** Definido como o valor máximo entre o peso real e o peso cúbico calculado (`max(peso_real, peso_cubado)`).

In [5]:
df['peso_cubado_g'] = calcular_peso_cubado(
    df['product_length_cm'], 
    df['product_height_cm'], 
    df['product_width_cm'], 
    fator_cubagem=6000
)

df['final_weight_g'] = np.maximum(df['product_weight_g'], df['peso_cubado_g'])

## Comprador e vendedor são do mesmo estado?

In [6]:
df['same_state'] = (df['customer_state'] == df['seller_state']).astype(int)

## Distância

Para estimar a distância de entrega, utilizamos a **Fórmula de Haversine**.

**Fórmula utilizada:**
$$d = 2r \arcsin\left(\sqrt{\sin^2\left(\frac{\Delta\phi}{2}\right) + \cos(\phi_1)\cos(\phi_2)\sin^2\left(\frac{\Delta\lambda}{2}\right)}\right)$$

Onde:
* $r$: Raio da Terra (~6371 km)
* $\phi$: Latitude (em radianos)
* $\lambda$: Longitude (em radianos)

In [7]:
df['distance_km'] = haversine_vectorized(
    df['lat_seller'].values, 
    df['lng_seller'].values, 
    df['lat_customer'].values, 
    df['lng_customer'].values,
    R=6371.0 # Raio da terra em KM
)

## Seleção de Features e Target

In [8]:
cols_finais = [
    'price',
    'final_weight_g',
    'volume_cm3',
    'product_density',
    'customer_state',
    'seller_state',
    'same_state',
    'lat_customer',
    'lng_customer',
    'lat_seller',
    'lng_seller',
    'distance_km',
    'freight_value'
]

df_final = df[cols_finais].copy()
df_final.head()

,price,final_weight_g,volume_cm3,product_density,customer_state,seller_state,same_state,lat_customer,lng_customer,lat_seller,lng_seller,distance_km,freight_value
0,58.90,650.0,3528.0,0.184240,RJ,SP,0,-21.762775,-41.309633,-22.496953,-44.127492,301.504681,13.29
1,239.90,30000.0,60000.0,0.500000,SP,SP,1,-20.220527,-50.903424,-23.565096,-46.518565,585.563937,19.93
2,199.00,3050.0,14157.0,0.215441,MG,MG,1,-19.870305,-44.593326,-22.262584,-46.171124,312.343511,17.87
3,12.99,400.0,2400.0,0.083333,SP,SP,1,-23.089925,-46.611654,-20.553624,-47.387359,293.168420,12.79
4,199.90,7000.0,42000.0,0.089286,SP,PR,0,-23.243402,-46.827614,-22.929384,-53.135873,646.163463,18.14


## Exportação dos Dados

In [9]:
df_final.to_csv(DATA_PROCESSED / "freight_prediction_abt.csv", index=False)